In [3]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd

# Define OAuth 2.0 scope (Read-only access to YouTube)
SCOPES = ["https://www.googleapis.com/auth/youtube.readonly"]

# Authenticate using OAuth 2.0
flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", SCOPES)
creds = flow.run_local_server(port=0)

# Build YouTube API client
youtube = build("youtube", "v3", credentials=creds)

# Function to get Uploads Playlist ID for the channel
def get_uploads_playlist_id():
    request = youtube.channels().list(
        part="contentDetails",
        id="UCVjCW3tx7MT24MikTJXOZIw"  # ✅ Fixed Channel ID
    )
    response = request.execute()

    # Ensure 'items' exists in response
    if "items" not in response or not response["items"]:
        raise Exception("❌ Error: No channel data found! Double-check the Channel ID.")

    return response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

# Function to fetch all videos (public & unlisted only)
def get_videos():
    playlist_id = get_uploads_playlist_id()
    videos = []
    next_page_token = None

    while True:
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=playlist_id,  # Fetch videos from Uploads playlist
            maxResults=50,  # Max per request
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response.get("items", []):
            title = item["snippet"]["title"]
            video_id = item["snippet"]["resourceId"]["videoId"]
            video_url = f"https://www.youtube.com/watch?v={video_id}"
            videos.append([title, video_url])

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return videos

# Fetch videos
video_list = get_videos()

# Save to Excel
df = pd.DataFrame(video_list, columns=["Title", "URL"])
df.to_excel("YouTube_Video_List.xlsx", index=False)
print("✅ YouTube video list saved as 'YouTube_Video_List.xlsx'")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=253406904901-shu0pjqhddobn21msfjn1o7ue5q3k020.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55713%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=AJfA5NT6L4nCl7L9pUOtyoDGl1u16B&access_type=offline
✅ YouTube video list saved as 'YouTube_Video_List.xlsx'
